In [33]:
#Q1
"""
We use our custom function to approximate the sine function.
"""

import torch
import math


class CustomFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors

        # grad_output: Tensor containing gradient of the loss with respect to the output.
        # computed the gradient of the loss with respect to the input which means bacward propagation.
        return grad_output * 0.5 * (15 * input ** 2 - 3)


dtype = torch.float
device = torch.device("cuda:0")

x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x) # We approximate this sine function.

# In our model, we have 4 weights to train: y = a + b * P3(c + d * x).
# These weights need to be initialized.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    P3 = CustomFunction.apply

    # Forward pass: predict y.
    # P3 using our custom backward function.
    y_pred = a + b * P3(c + d * x)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
    
    # Use autograd to compute the backward pass.
    loss.backward()
 
    # Update weights using gradient descent
    with torch.no_grad():
   
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')

99 209.9583282470703
199 144.66018676757812
299 100.70249938964844
399 71.03519439697266
499 50.97850799560547
599 37.403133392333984
699 28.206865310668945
799 21.97318458557129
899 17.7457275390625
999 14.877889633178711
1099 12.931764602661133
1199 11.610918045043945
1299 10.714248657226562
1399 10.105474472045898
1499 9.692106246948242
1599 9.411375045776367
1699 9.220745086669922
1799 9.091285705566406
1899 9.003361701965332
1999 8.943639755249023
Result: y = 1.106413745344259e-11 + -2.208526849746704 * P3(-6.452179068805464e-11 + 0.2554861009120941 x)


In [14]:
#Q2

"""
We will implement many custom kernels. Try to improve the classification accuracy and F-1 scores.
"""

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import sklearn
import pandas as pd
import numpy as np
from google.colab import files
import io

# Upload data
uploaded = files.upload()


# load data using pandas and cast them to numpy
X = pd.read_csv(io.BytesIO(uploaded['trainX.csv']),header = None)
Y = pd.read_csv(io.BytesIO(uploaded['trainY.csv']),header = None)
XTe = pd.read_csv(io.BytesIO(uploaded['testX.csv']),header = None)
YTe = pd.read_csv(io.BytesIO(uploaded['testY.csv']),header = None)
X = (X.to_numpy())
Y = (Y.to_numpy())
XTe = (XTe.to_numpy())
YTe = (YTe.to_numpy())


Saving testX.csv to testX (1).csv
Saving testY.csv to testY (1).csv
Saving trainX.csv to trainX (1).csv
Saving trainY.csv to trainY (1).csv


In [34]:
# Custom kernels that I implemented

def euclid_distance(X1, X2):

    # It have two parameter for each dataset.
    # It returns Euclid Distance between two dataset matrixes
    term1 = (X1 ** 2).sum(axis=1).reshape(-1,1)
    term2 = (X2 ** 2).sum(axis=1)
    term3 = -2 * np.dot(X1,X2.T)
    return np.abs(term1 + term2 - term3)


def linear_kernel(X1, X2):
    #  Linear kernel : k (xi , xj) = xi . xj 
    return X1.dot(X2.T)

def poly_kernel_homo(X1,X2,degree=2):
    #Polynomial (homogeneous) : k(xi,xj) = (xi . xj)^d
    return (X1.dot(X2.T)) ** degree

def poly_kernel_inhomo(X1,X2,degree=2, c=10):
    #Polynomial (homogeneous) : k(xi,xj) = (xi . xj + c)^d
    return (X1.dot(X2.T) + c) ** degree

def rbf_kernel(X1,X2,gamma = 0.1):
    #  Gaussian radial basis function (rbf) : k(xi, yj) = e^(-gamma||xi - yj||^2)
    return np.exp(-gamma * euclid_distance(X1,X2))


def cosine_kernel(X1,X2):
    # Cosine :  K(xi, xj) = xi.xj / (||x|| . ||y||)
    term1 = np.sqrt((X1 ** 2).sum(axis=1)).reshape(-1, 1)
    term2 = np.sqrt((X2 ** 2).sum(axis=1)).reshape(-1, 1)
    return X1.dot(X2.T) / (term1.dot(term2.T))

def multiquadric_kernel(X1,X2,c=2):
    # multiquadric :    k(xi, yj) = 1 / sqrt(|| xi- yj ||^2 + c^2)
    return np.sqrt(euclid_distance(X1,X2) + c ** 2)

def cauchy_kernel(X1,X2, sigma= 2):
    # cauchy : k(xi, xj) = 1 / (1 + || xi - xj ||^2 / sigma ^ 2)

    distance = euclid_distance(X1, X2)
    return 1 / (1 + distance / sigma)

def tstudent_kernel(X1,X2, d=2):
    # tstudent :  k(xi , xj) = 1 / (1 + ||xi - yj||^d)

    return 1 / (1 + (euclid_distance(X1,X2) ** d / 2))

def log_kernel(X1,X2, d=2):
    #  k(xi, xj) = -log(|| xi-xj ||^d) + 1 

    return -np.log(euclid_distance(X1, X2) ** d / 2.) + 1

def thin_plate_kernel(X1,X2,n=3):
    # thin-plate: k(xi, xj) = || xi-xj ||^2n+1
    return euclid_distance(X1, X2) ** ((2*n+1) / 2)


In [46]:

Y= Y.reshape(160,)
# apply custom kernels to SVM

clf = SVC()
clf = SVC(kernel = linear_kernel, random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Linear Kernel DIY")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

clf = SVC()
clf = SVC(kernel = 'linear', random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Linear Kernel sklearn version")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

clf = SVC(kernel = poly_kernel_homo, random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Homo Polynomial Kernel DIY")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

clf = SVC(kernel = 'poly', gamma=1,degree=2, coef0=0, random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Homo Polynomial Kernel sklearn version")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

clf = SVC(kernel = poly_kernel_inhomo, random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Inhomo Polynomial Kernel DIY")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

clf = SVC(kernel = "poly",gamma= 1, degree = 2, coef0= 10, random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Inhomo Polynomial Kernel sklearn version")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

clf = SVC(kernel = rbf_kernel, random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Gaussian Radial Basis Function Kernel DIY version")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

clf = SVC(kernel = 'rbf', gamma = 0.1, random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Gaussian Radial Basis Function Kernel sklearn version")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

clf = SVC(kernel = cosine_kernel, random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Cosine Kernel")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

clf = SVC(kernel = multiquadric_kernel, random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Multiquadric Kernel")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

clf = SVC(kernel = cauchy_kernel, random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Cauchy Kernel")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

clf = SVC(kernel = tstudent_kernel, random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Tstudent Kernel")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

clf = SVC(kernel = log_kernel, random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Log Kernel")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

clf = SVC(kernel = thin_plate_kernel, random_state=2011)
clf.fit(X, Y)
yp = clf.predict(XTe)
print("Thin Plate Kernel")
print("AcuuracyScore: ", accuracy_score(YTe, yp))
print("F1 score: ", f1_score(YTe, yp, average='macro'))
print()

# # The version of sklearn should be "0.22.2.post1" for reproducibility.
print(sklearn.__version__)

Linear Kernel DIY
AcuuracyScore:  0.375
F1 score:  0.323867478025693

Linear Kernel sklearn version
AcuuracyScore:  0.375
F1 score:  0.323867478025693

Homo Polynomial Kernel DIY
AcuuracyScore:  0.725
F1 score:  0.6925227113906359

Homo Polynomial Kernel sklearn version
AcuuracyScore:  0.725
F1 score:  0.6925227113906359

Inhomo Polynomial Kernel DIY
AcuuracyScore:  0.75
F1 score:  0.7252747252747254

Inhomo Polynomial Kernel sklearn version
AcuuracyScore:  0.75
F1 score:  0.7252747252747254

Gaussian Radial Basis Function Kernel DIY version
AcuuracyScore:  0.65
F1 score:  0.6419437340153452

Gaussian Radial Basis Function Kernel sklearn version
AcuuracyScore:  0.625
F1 score:  0.5943204868154158

Cosine Kernel
AcuuracyScore:  0.5
F1 score:  0.494949494949495

Multiquadric Kernel
AcuuracyScore:  0.425
F1 score:  0.3943383805134958

Cauchy Kernel
AcuuracyScore:  0.55
F1 score:  0.53125

Tstudent Kernel
AcuuracyScore:  0.575
F1 score:  0.5682539682539682

Log Kernel
AcuuracyScore:  0.5
F